In [33]:
import pandas as pd 
import numpy as np
import xml.etree.ElementTree as et 
import os
import regex as re
import json
import os
import glob
from tqdm.autonotebook import tqdm

In [60]:
groovylist = []
for root, dirs, files in os.walk("../../../Downloads/PostNLQAS"):
    groovylist = groovylist + glob.glob(root+'/*.groovy')
    groovylist = groovylist + glob.glob(root+'/*.gsh')

In [61]:
comments = r'(\/\*[ -}\n\s]*?\*\/)'
comments2 = r'\/\/.+'
defines = r'def.?{\n[ -z\n\t\;\=\.\(]*\}'
imports = r'import.*?\n]'
linecount = r'\n'
whitelines = r'^([\n\s]*)$' #multiline
commentcount = r'\/\/.*?\n'
reglist = [{'regex':comments, 'flag':False, 'name':"comments"},
#           {'regex':functions, 'flag':False, 'name':"defines"},
#           {'regex':imports, 'flag':False, 'name':"imports"},
#           {'regex':linecount, 'flag':False, 'name':"linecount"},
          {'regex':whitelines, 'flag':True, 'name':"whitelines"}]

In [62]:
data = {"filepath":[],"comments":[],"singlelinecomments":[],"defines":[],"imports":[],"linecount":[],"whitelines":[],"linesofcode":[]}
for filename in groovylist:
    with open(filename, 'r') as file:
        filecontent = file.readlines()
        linesum = sum(1 for line in filecontent)
        filestrcontent = str(filecontent)
#         linesum += len(filecontent)
        importcount = filestrcontent.count("import")
        defcount = filestrcontent.count("def")
        singlelinecommentcount = filestrcontent.count("//")
        for reg in reglist:
#             print(reg)
            match = None
            if reg['flag']:
                match = []
                for line in filecontent:
                    match += re.findall(r'^([\n\s]*)$', line)
#                 print(match)
            else:
                match = re.findall(reg['regex'], filestrcontent)
            if not match == []:
                if reg['flag']:
                    data[reg['name']].append(len(match))
                else:
                    if reg['name'] == "comments":
                        res = re.search(reg['regex'],filestrcontent)
                        match = str(res.group(1))
                        match = ' '.join([str(elem) for elem in match.split("', '")])
                        match = re.sub(r'\/\/.+?\\n', " ", match)
                        match = ' '.join([str(elem) for elem in match.split("', '")])
                        match = re.sub(r'\\n \\n', "\n", match)
                        match = ' '.join([str(elem) for elem in match.split("', '")])
                        match = re.sub(r'\\n', "\n", match)
                        match = re.findall(r'\n',match)
                    else:
                        match = match[0].split(',')
        #                 print(len(match.groups()))
                    data[reg['name']].append(len(match))
            else:
                data[reg['name']].append(0)
        data['filepath'].append(filename)
        data['imports'].append(importcount)
        data['defines'].append(defcount)
        data['linecount'].append(linesum)
        data['singlelinecomments'].append(singlelinecommentcount)
#         data['linesofcode'].append((data['linecount'][-1]-data['comments'][-1]-data['whitelines'][-1]-data['singlelinecomments'][-1]))
#         print(len(filecontent))
# print(data)

In [63]:
for i in range(0,len(data['imports'])):
    data['linesofcode'].append((data['linecount'][i]-data['comments'][i]-data['whitelines'][i]-data['singlelinecomments'][i]))
# print(data)

In [64]:
# groovylist[37]

In [65]:
# print(data['linecount'][37],data['comments'][37],data['whitelines'][37],data['singlelinecomments'][37])

In [66]:
df = pd.DataFrame(data)

In [1]:
# df.head()

Hypotesis, Lines of code and imports are high indicators for the error rate

In [10]:
df.to_csv("../generateddata/qasgroovy.csv")

# Metrics

In [16]:
import pandas as pd 
import xml.etree.ElementTree as et 

prefix = "../../../Downloads/FinalQASMetrics/"

metricslist = []
df_cols = ["metric","file", "line","problem_class","problem_severity","description"]
rows = []
for root, dirs, files in os.walk(prefix):
    for i in files:
        print(i)
        metric = i
        xtree = et.parse(prefix+i)
        problems = xtree.getroot() 
        for node in list(problems): 
            file = node.find("file").text if node is not None else None
            line = node.find("line").text if node is not None else None
            package = node.find("package").text if node is not None else None
            problem_class = node.find("problem_class").text if node is not None else None
            problem_severity = node.find("problem_class").attrib.get("severity") if node is not None else None
            description = node.find("description").text if node is not None else None

            file = file.replace("file://$PROJECT_DIR$", "../../../Downloads/PostNLQAS")
            rows.append({"metric":metric,"file": file, "line": line, 
                        "problem_class": problem_class,
                        "problem_severity":problem_severity,"description":description})
        

out_df = pd.DataFrame(rows)

NestedBlockDepth.xml
ParameterCount.xml
GroovyOverlyComplexMethod.xml
AbcMetric.xml


In [53]:
def abcscore(x):
    regexexpression = r".+?([\d\.]{3,5}).+"
    res = re.search(regexexpression, x)
    if res:
        return res.group(1)
    return None

def cyclomaticcomplexity(x):
    regexexpression = r"\=(\d{1,2})+"
    res = re.search(regexexpression, x)
    if res:
        return res.group(1)
    return None

def nestedblock(x):
    regexexpression = r".+?([\d]+)"
    res = re.search(regexexpression, x)
    if res:
        return res.group(1)
    return None

def ParameterCount(x):
    regexexpression = r".+?([\d]+).+"
    res = re.search(regexexpression, x)
    if res:
        return res.group(1)
    return None

def formatter(metric, x):
    if metric == 'NestedBlockDepth.xml':
        return nestedblock(x)
    elif metric == 'ParameterCount.xml':
        return ParameterCount(x)
    elif metric == 'GroovyOverlyComplexMethod.xml':
        return cyclomaticcomplexity(x)
    elif metric == 'AbcMetric.xml':
        return abcscore(x)

In [2]:
# out_df.describe()

In [3]:
nested_df = out_df[out_df['metric']=="NestedBlockDepth.xml"]
nested_df['nestedscore'] = nested_df.apply(lambda row: formatter(row['metric'],row['description']), axis=1)
nested_df = nested_df.drop(columns=["metric", "line", "problem_severity", "description", "problem_class"])
for file in nested_df['file'].unique():
    values = nested_df[nested_df['file']==file].nestedscore.values
    val = np.amax(values)
    nested_df.loc[nested_df['file'] == file, ['nestedscore']] = val
nested_df = nested_df.drop_duplicates()
nested_df.head()

In [4]:
param_df = out_df[out_df['metric']=="ParameterCount.xml"]
param_df['paramscore'] = param_df.apply(lambda row: formatter(row['metric'],row['description']), axis=1)
param_df = param_df.drop(columns=["metric", "line", "problem_severity", "description", "problem_class"])
for file in param_df['file'].unique():
    values = param_df[param_df['file']==file].paramscore.values
    val = np.amax(values)
    param_df.loc[param_df['file'] == file, ['paramscore']] = val
param_df = param_df.drop_duplicates()
param_df.head()

In [5]:
cyclo_df = out_df[out_df['metric']=="GroovyOverlyComplexMethod.xml"]
cyclo_df['cycloscore'] = cyclo_df.apply(lambda row: formatter(row['metric'],row['description']), axis=1)
cyclo_df = cyclo_df.drop(columns=["metric", "line", "problem_severity", "description", "problem_class"])
for file in cyclo_df['file'].unique():
    values = cyclo_df[cyclo_df['file']==file].cycloscore.values
    val = np.amax(values)
    cyclo_df.loc[cyclo_df['file'] == file, ['cycloscore']] = val
cyclo_df = cyclo_df.drop_duplicates()
cyclo_df.head()

In [42]:
abc_df = out_df[out_df['metric']=="AbcMetric.xml"]
abc_df['abcscore'] = abc_df.apply(lambda row: formatter(row['metric'],row['description']), axis=1)
abc_df = abc_df.drop(columns=["metric", "line", "problem_severity", "description", "problem_class"])
for file in abc_df['file'].unique():
    values = abc_df[abc_df['file']==file].abcscore.values
    val = np.amax(values)
    abc_df.loc[abc_df['file'] == file, ['abcscore']] = val
cyclo_df = cyclo_df.drop_duplicates()
abc_df.head()

In [6]:
out_df['description'][29]
df = df.rename(mapper={"filepath":"file"}, axis="columns")
df.head()

In [7]:
finaldf = pd.concat([df, abc_df, cyclo_df, param_df, nested_df])
finaldf = finaldf.fillna(value=0)
finaldf.head()

In [82]:
finaldf.to_csv("../generateddata/qasgroovymetrics.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805 entries, 0 to 265
Data columns (total 12 columns):
abcscore              644 non-null object
comments              1768 non-null float64
cycloscore            161 non-null object
defines               1768 non-null float64
file                  2805 non-null object
imports               1768 non-null float64
linecount             1768 non-null float64
linesofcode           1768 non-null float64
nestedscore           45 non-null object
paramscore            187 non-null object
singlelinecomments    1768 non-null float64
whitelines            1768 non-null float64
dtypes: float64(7), object(5)
memory usage: 284.9+ KB
